In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=c23492f11dd00850f9372da7bc61b5923740037ad322cf715ac1628648abb045
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [5]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Accident').getOrCreate()

from pyspark.sql.types import *
from pyspark.sql.functions import *

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df=spark.read.csv('drive/My Drive/BDA-SPARK/Road_accidents_time.csv',header=True,inferSchema=True)
df.show(10)

+-------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
|     STATE/UT|YEAR|0-3 hrs. (Night)|3-6 hrs. (Night)|6-9 hrs (Day)|9-12 hrs (Day)|12-15 hrs (Day)|15-18 hrs (Day)|18-21 hrs (Night)|21-24 hrs (Night)|
+-------------+----+----------------+----------------+-------------+--------------+---------------+---------------+-----------------+-----------------+
|A & N Islands|2001|               2|               6|           29|            40|             39|             40|               18|                7|
|A & N Islands|2002|               2|               6|           22|            41|             33|             33|               23|                8|
|A & N Islands|2003|               2|               8|           31|            35|             28|             36|               25|               15|
|A & N Islands|2004|               2|               5|           29|            42|     

In [8]:
df1=spark.read.csv('drive/My Drive/BDA-SPARK/Road_accidents.csv',header=True,inferSchema=True)
df1.show(10)

+-------------+----+-------+--------+-----+-----+---+----+----+------+---------+-------+--------+--------+
|     STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|
+-------------+----+-------+--------+-----+-----+---+----+----+------+---------+-------+--------+--------+
|A & N Islands|2001|      8|      23|   15|   15| 14|  19|  14|    19|        7|     12|      13|      22|
|A & N Islands|2002|     12|      10|   14|   16| 10|   7|  16|    11|       23|     21|      11|      17|
|A & N Islands|2003|     19|      13|   15|   13| 13|  12|   8|    16|       17|     25|      14|      15|
|A & N Islands|2004|     21|      14|   22|   17| 13|  18|  16|    19|       16|     20|      15|      24|
|A & N Islands|2005|     19|      21|   22|   17| 13|  19|  21|    14|       15|     19|      10|      16|
|A & N Islands|2006|     21|      13|    4|   22|  9|  14|  12|    14|        8|     14|       6|      18|
|A & N Islands|2007|     17|      16|

In [9]:
# 1. Add column 'Total', giving sum of accidents in each month
total = df1.withColumn('TOTAL',(col('JANUARY')+col('FEBRUARY')+col('MARCH')+col('APRIL')+col('MAY')\
    +col('JUNE')+col('JULY')+col('AUGUST')+col('SEPTEMBER')+col('OCTOBER')+col('NOVEMBER')+col('DECEMBER')))
total.show()


+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
|      STATE/UT|YEAR|JANUARY|FEBRUARY|MARCH|APRIL| MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|TOTAL|
+--------------+----+-------+--------+-----+-----+----+----+----+------+---------+-------+--------+--------+-----+
| A & N Islands|2001|      8|      23|   15|   15|  14|  19|  14|    19|        7|     12|      13|      22|  181|
| A & N Islands|2002|     12|      10|   14|   16|  10|   7|  16|    11|       23|     21|      11|      17|  168|
| A & N Islands|2003|     19|      13|   15|   13|  13|  12|   8|    16|       17|     25|      14|      15|  180|
| A & N Islands|2004|     21|      14|   22|   17|  13|  18|  16|    19|       16|     20|      15|      24|  215|
| A & N Islands|2005|     19|      21|   22|   17|  13|  19|  21|    14|       15|     19|      10|      16|  206|
| A & N Islands|2006|     21|      13|    4|   22|   9|  14|  12|    14|        

In [10]:
# 2.Which state has highest number of accidents in year 2013?
high_acc = total.select('STATE/UT','YEAR','TOTAL').filter(col('YEAR') == 2013)
high_acc.where(col('TOTAL') == high_acc.agg({'TOTAL':'max'}).collect()[0][0]).show()

+----------+----+-----+
|  STATE/UT|YEAR|TOTAL|
+----------+----+-----+
|Tamil Nadu|2013|66238|
+----------+----+-----+



In [13]:
# 3.Find the average monthly accidents for each state.
monthAcc = df1.withColumn('AVG',(col('JANUARY')+col('FEBRUARY')+col('MARCH')+col('APRIL')+col('MAY')+col('JUNE')+col('JULY')+col('AUGUST')
    +col('SEPTEMBER')+col('OCTOBER')+col('NOVEMBER')+col('DECEMBER'))/12)
monthAcc.select('STATE/UT','AVG').groupBy('STATE/UT','AVG').count().distinct().show(30)

+----------------+------------------+-----+
|        STATE/UT|               AVG|count|
+----------------+------------------+-----+
|       Rajasthan|           1714.25|    1|
|  Madhya Pradesh|           2547.75|    1|
|         Manipur|44.833333333333336|    1|
|           Assam| 304.6666666666667|    1|
|     West Bengal|1205.6666666666667|    1|
|      Tamil Nadu|            4331.5|    1|
|       Karnataka|3765.8333333333335|    1|
|  Madhya Pradesh|2836.1666666666665|    1|
|        Nagaland|               3.5|    1|
|         Tripura| 68.16666666666667|    1|
|     West Bengal|1090.4166666666667|    1|
|    D & N Haveli| 7.583333333333333|    1|
|         Haryana|            872.75|    1|
|       Karnataka|3851.6666666666665|    1|
|         Haryana|            952.25|    1|
|          Kerala|            3014.5|    1|
|      Puducherry|120.91666666666667|    1|
|       Rajasthan|2025.1666666666667|    1|
|   A & N Islands|17.166666666666668|    1|
|       Rajasthan|1666.583333333

In [15]:
# 4. Which month has highest accidents?
highest_acc = df1.select([col for col in df1.columns if col!= 'STATE/UT' and col!='YEAR'])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'max'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()

+-----+-----+
|month|value|
+-----+-----+
|  MAY| 9330|
+-----+-----+



In [16]:
# 6.Which state has more accidents from year 2001 to 2014?
more_acc = total.select('TOTAL','YEAR','STATE/UT').where(col('YEAR') <= 2014)
more_accident = more_acc.groupBy('STATE/UT','TOTAL').count().agg({'TOTAL':'sum'}).collect()[0][0]
more_accident

5599501

In [19]:
# 8. Which month has more accidents over the year?
highest_acc = df1.select([col for col in df1.columns if col!= 'STATE/UT' and col!='YEAR'])
var  = {}
for i in highest_acc.columns:
  var[i] = highest_acc.agg({i:'max'}).collect()[0][0]
df4 = spark.createDataFrame(var.items(), ["month", "value"])
maxValue = df4.agg({'value':'max'}).collect()[0][0]
df4.select('month','value').where(col('value')==maxValue).show()

+-----+-----+
|month|value|
+-----+-----+
|  MAY| 9330|
+-----+-----+

